In [13]:
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from matplotlib import pyplot as plt
from keras import optimizers
from keras.models import Sequential
from keras import layers
from keras import regularizers
from tensorflow.keras import backend as K
from physionet_generator import DataGenerator
from physionet_processing import spectrogram

In [14]:
# Data paths
data_root = os.path.normpath("C:\\Users\\nazrulsa\\Desktop\\Data-Fusion\\")
records_file = os.path.join('.', 'records_filtered.csv')

# Load records
records_df = pd.read_csv(records_file, names=['subject_id', 'study_id', 'icd_code', 'icd_title', 'path'])

# Label encoding
label_set = list(sorted(records_df['icd_title'].unique()))
encoder = LabelEncoder().fit(label_set)
records_df['encoded'] = encoder.transform(records_df['icd_title'])

# Split dataset
test_split = 0.33
idx = np.arange(records_df.shape[0])
id_train, id_val = train_test_split(idx, test_size=test_split, shuffle=True, random_state=123)

partition = {
    'train': list(records_df.iloc[id_train]['study_id']),
    'validation': list(records_df.iloc[id_val]['study_id']),
}

labels = dict(zip(records_df['study_id'], records_df['encoded']))


In [17]:
print(f"Shape of ECG data: {ecg_data.shape}")


Shape of ECG data: (5000, 12)


In [20]:
# Parameters for spectrogram
spectrogram_nperseg = 64  # Window length
spectrogram_noverlap = 32  # Overlap length
n_classes = len(label_set)  # Number of classes
batch_size = 32  # Batch size

# Check ECG data shape
print(f"ECG data shape: {ecg_data.shape}")  # Should be (5000, 12)

# Initialize list for spectrograms
spectrograms = []

# Iterate through all 12 leads
for lead in range(12):
    try:
        lead_data = ecg_data[:, lead]  # Extract data for the current lead
        if lead_data.ndim != 1:  # Ensure it's 1D
            raise ValueError(f"Lead {lead} data is not 1D. Shape: {lead_data.shape}")
        
        # Compute the spectrogram
        f, t, Sxx = spectrogram(
            lead_data,
            fs=500,  # Sampling frequency for MIMIC-IV data
            nperseg=spectrogram_nperseg,
            noverlap=spectrogram_noverlap
        )
        spectrograms.append(Sxx)
    except Exception as e:
        print(f"Error calculating spectrogram for lead {lead}: {e}")
        continue

# Validate spectrogram generation
if len(spectrograms) == 0:
    raise RuntimeError("No valid spectrograms were generated.")

# Spectrogram dimensions for one lead
dim = spectrograms[0].shape  # Time steps and frequency bins for one lead

# Stack spectrograms for all leads
stacked_spectrogram = np.stack(spectrograms, axis=-1)  # Shape: (time_steps, frequency_bins, 12)

# Output dimension details
print(f"Spectrogram dimensions for one lead: {dim}")
print(f"Stacked spectrogram dimensions (time_steps, frequency_bins, 12): {stacked_spectrogram.shape}")
print(f"Number of classes: {n_classes}")


ECG data shape: (5000, 12)
Error calculating spectrogram for lead 0: spectrogram() got an unexpected keyword argument 'fs'
Error calculating spectrogram for lead 1: spectrogram() got an unexpected keyword argument 'fs'
Error calculating spectrogram for lead 2: spectrogram() got an unexpected keyword argument 'fs'
Error calculating spectrogram for lead 3: spectrogram() got an unexpected keyword argument 'fs'
Error calculating spectrogram for lead 4: spectrogram() got an unexpected keyword argument 'fs'
Error calculating spectrogram for lead 5: spectrogram() got an unexpected keyword argument 'fs'
Error calculating spectrogram for lead 6: spectrogram() got an unexpected keyword argument 'fs'
Error calculating spectrogram for lead 7: spectrogram() got an unexpected keyword argument 'fs'
Error calculating spectrogram for lead 8: spectrogram() got an unexpected keyword argument 'fs'
Error calculating spectrogram for lead 9: spectrogram() got an unexpected keyword argument 'fs'
Error calcula

RuntimeError: No valid spectrograms were generated.

: 